# Equilibria Along Parameters

In [1]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from scipy.special import lambertw
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.colors
import warnings
import pickle
from matplotlib.colors import LinearSegmentedColormap
import ipynb.fs.defs.Functions_Equations_Equilibrium_Simulation as baseFuncs
import ipynb.fs.defs.Functions_Behavioral_Stability as stabFuncs

### Slice of Parameter

In [2]:
def paramSlice(param_label, lower, upper, para, param_mesh = 500, stab_slice = False, init_mesh = 10, xtol = 1e-8):
    param_vary = np.linspace(lower, upper, param_mesh)
    param_positions = []
    stab_cat_Rs = []
    
    
    # Finding all equilibrium positions
    for param_it, param_val in enumerate(param_vary):
        # change para
        para[param_label] = param_val

        # Evaluate from a grid based on maximum values of Lambert W branch 
        if para['b']  == 0:
            init_Ru_vals = list(np.linspace(0, para['w_max']/para['k_A'], init_mesh)) #Starts at 0 to include bloat position. 
            init_Ro_vals = list(np.linspace(1, para['w_max']/para['k_A'], init_mesh)) #Starts at 1 to avoid division by zero total rules.
        elif para['b'] < np.exp(-1):
            init_Ru_vals = list(np.linspace(1, 1501, init_mesh))
            init_Ro_vals = list(np.geomspace(1, -2*para['w_max']*np.real(lambertw(-para['b'], -1))/(para['k_A']*para['b']) + 1, init_mesh))
            init_Ro_vals.append(-1.05*para['w_max']*np.real(lambertw(-para['b'], -1))/(para['k_A']*para['b']) + 1) 
            # We could manually append to ensure the branch position is in the list as the upper values are more spaced out.
        else:
            init_Ru_vals = list(np.linspace(1, 1501, init_mesh))
            init_Ro_vals = list(np.geomspace(1, -10*para['w_max']*np.real(lambertw(-np.exp(-0.9999), -1))/(para['k_A']*np.exp(-0.9999)) + 1, init_mesh))
        init_lists = [init_Ru_vals, init_Ro_vals]
        x0s = [[x,y] for x in init_lists[0] for y in init_lists[1]]
        
        # evaluate from many initial conditions and save convergent positions
        positions, flag = baseFuncs.equilFuncBehavioral(para, all_results = True, x0s = x0s)
        if len(positions) == 0:
            print(f"No solutions found for parameter position. {param_val}")
            param_positions.append([[np.nan, np.nan]])
            stab_cat_Rs.append([np.nan, np.nan])
        else:
            positions = sorted(positions)
            param_positions.append(positions)
            stab_cat_Rs.append(positions)
        
            # Also initialize from between the solutions for the next iterations
            for pos_iter in range(min(10, param_it)):
                for i in range(len(param_positions[param_it-pos_iter])-1):
                    betweenPosit = (np.asarray(param_positions[param_it-pos_iter][i])+np.asarray(param_positions[param_it-pos_iter][i+1]))/2
                    x0s = x0s + list(betweenPosit)
                    x0s = x0s +[param_positions[param_it-pos_iter][i][0], param_positions[param_it-pos_iter][i][1]]
                x0s = x0s + [param_positions[param_it-pos_iter][-1][0], param_positions[param_it-pos_iter][-1][1]]

    # Find number of branches and initialize data containers for plotting
    branch_num = len(max(param_positions, key=len))
     # Type 1 plot data (show_rules = True)
    sum_Rs = np.ones([branch_num, param_mesh])
    sum_Rs[:] = np.nan
    Rus = np.ones([branch_num, param_mesh])
    Rus[:] = np.nan
    Ros = np.ones([branch_num, param_mesh])
    Ros[:] = np.nan
     # Bureaucratic metrics
    prop_Ro = np.ones([branch_num, param_mesh])
    prop_Ro[:] = np.nan
    prop_w_A = np.ones([branch_num, param_mesh])
    prop_w_A[:] = np.nan
    prop_w_A_star = np.ones([branch_num, param_mesh])
    prop_w_A_star[:] = np.nan
     # Allocation coefficients
    ps = np.ones([branch_num, param_mesh])
    ps[:] = np.nan
    cs = np.ones([branch_num, param_mesh])
    cs[:] = np.nan
    ds = np.ones([branch_num, param_mesh])
    ds[:] = np.nan
    # Utility
    utilities = np.ones([branch_num, param_mesh])
    utilities[:] = np.nan
     # Both type stability data
    stabs = np.ones([branch_num, param_mesh])
    stabs[:] = np.nan
    stable_nontriv = np.ones([branch_num, param_mesh])
    stable_nontriv[:] = 0
    stable_bloat = np.ones([branch_num, param_mesh])
    stable_bloat[:] = 0


    # Initializing eigenvalue storage
    eigs_list = []
    for i in range(branch_num):
        eigs_list.append([])

    # Getting the stability of all equilibrium positions and appropriately storing the data
    stab_cat = []
    for param_idx, positions_it in enumerate(param_positions):
        para[param_label] = param_vary[param_idx]

        # Fill remained of cells with nan values
        while len(positions_it) < branch_num:
            positions_it.append((np.nan, np.nan))

        if param_idx == 0:
            for initial_it, initial_position in enumerate(positions_it):
                if np.isfinite(initial_position[0]) and np.isfinite(initial_position[1]):
                    Rus[initial_it, 0] = initial_position[0]
                    Ros[initial_it, 0] = initial_position[1]
                    prop_Ro[initial_it, 0] = initial_position[1]/np.sum(initial_position) 
                    sum_Rs[initial_it, 0] = np.sum(initial_position)
                    stab, eig = stabFuncs.jacobian_eignvalues((initial_position[0], initial_position[1]), para)
                    stabs[initial_it, 0] = stab #stability function here
                    if initial_position[0] > 0 and stab:
                        stable_nontriv[initial_it, 0] = 1
                    if initial_position[0] == 0 and stab:
                        stable_bloat[initial_it, 0] = 1
                    output = baseFuncs.behavioral_work_eqns((initial_position[0], initial_position[1]), para, True, True)
                    prop_w_A[initial_it, 0] = output[0]/para['w_max']
                    prop_w_A_star[initial_it, 0] = output[7]/para['w_max']
                    ps[initial_it, 0] = output[6]
                    utilities[initial_it, 0] = para['A']*((output[1])**(para['alpha']))*((initial_position[0])**(para['beta']))
                    
        else:
            last_positions_it = list(zip(list(Rus[:, param_idx-1]), list(Ros[:, param_idx-1]))) #Get last positions with their indices
            
            # If increased number of positions
            if np.count_nonzero(~np.isnan(positions_it)) >= np.count_nonzero(~np.isnan(last_positions_it)): #If number of filled positions is larger now.
                # We need to keep track of which indices (in the new positions) are not matched to a closest previous.
                unused = np.arange(branch_num)
                # Remove new positions which are nan
                unused = np.delete(unused, np.argwhere(np.isnan(list(Rus[:, param_idx])) == True))
            
                # For each of the previous positions, find the new position which is closest to it and write the data into the same track.
                for last_order_idx_it, last_position_it in enumerate(last_positions_it): #for each of the previous equilibria
                    if np.isfinite(last_position_it[0]) and np.isfinite(last_position_it[1]): #
                        closest_new_position = min(enumerate(positions_it), key=lambda x: np.sqrt((last_position_it[0] - x[1][0])**2 + (last_position_it[1] - x[1][1])**2))
                        unused = np.delete(unused, np.argwhere(unused==closest_new_position[0]))
                        Rus[last_order_idx_it, param_idx] = closest_new_position[1][0]
                        Ros[last_order_idx_it, param_idx] = closest_new_position[1][1]
                        prop_Ro[last_order_idx_it, param_idx] = closest_new_position[1][1]/np.sum(closest_new_position[1]) 
                        sum_Rs[last_order_idx_it, param_idx] = np.sum(closest_new_position[1])
                        stab, eig = stabFuncs.jacobian_eignvalues((closest_new_position[1][0], closest_new_position[1][1]), para)
                        stabs[last_order_idx_it, param_idx] = stab #stability function here
                        if closest_new_position[1][0] > 0 and stab:
                            stable_nontriv[last_order_idx_it, param_idx] = 1
                        if closest_new_position[1][0] == 0 and stab:
                            stable_bloat[last_order_idx_it, param_idx] = 1
                        output = baseFuncs.behavioral_work_eqns((closest_new_position[1][0], closest_new_position[1][1]), para, True, True)
                        prop_w_A[last_order_idx_it, param_idx] = output[0]/para['w_max']
                        prop_w_A_star[last_order_idx_it, param_idx] = output[7]/para['w_max']
                        ps[last_order_idx_it, param_idx] = output[6]
                        utilities[last_order_idx_it, param_idx] = para['A']*((output[1])**(para['alpha']))*((closest_new_position[1][0])**(para['beta']))
                        
                # Find indices that are empty and have been empty for all time so far. Call this list empty_idx
                empty_idx = []
                for i in range(branch_num):
                    if np.count_nonzero(~np.isnan(Rus[i, :])) == 0:
                        empty_idx.append(i)
                #print(unused, empty_idx, positions_it, last_positions_it)
                        
                # For all positions that were not matched to a previous position, assign them to the unused tracks.
                for it, indx in enumerate(unused):
                    Rus[empty_idx[it], param_idx] = positions_it[indx][0] 
                    Ros[empty_idx[it], param_idx] = positions_it[indx][1] 
                    prop_Ro[empty_idx[it], param_idx] = positions_it[indx][1]/np.sum(positions_it[indx]) 
                    sum_Rs[empty_idx[it], param_idx] = np.sum(positions_it[indx])
                    stab, eig = stabFuncs.jacobian_eignvalues((positions_it[indx][0], positions_it[indx][1]), para)
                    stabs[empty_idx[it], param_idx] = stab #Stability function that can handle nans
                    if positions_it[indx][0] > 0 and stab:
                        stable_nontriv[empty_idx[it], param_idx] = 1
                    if positions_it[indx][0] == 0 and stab:
                        stable_bloat[empty_idx[it], param_idx] = 1
                    output = baseFuncs.behavioral_work_eqns((positions_it[indx][0], positions_it[indx][1]), para, True, True)
                    prop_w_A[empty_idx[it], param_idx] = output[0]/para['w_max']
                    prop_w_A_star[empty_idx[it], param_idx] = output[7]/para['w_max']
                    ps[empty_idx[it], param_idx] = output[6]
                    utilities[empty_idx[it], param_idx] = para['A']*((output[1])**(para['alpha']))*((positions_it[indx][1])**(para['beta']))
            
            # If number of new positions is less than the previous number of positions
            if np.count_nonzero(~np.isnan(positions_it)) < np.count_nonzero(~np.isnan(last_positions_it)): #If number of filled positions is larger now.
                # For each of the previous positions, find the new position which is closest to it and write the data into the same track.
                for order_idx_it, position_it in enumerate(positions_it): #for each of the new equilibria
                    if np.isfinite(position_it[0]) and np.isfinite(position_it[1]): 
                        closest_last_position = min(enumerate(last_positions_it), key=lambda x: np.sqrt((position_it[0] - x[1][0])**2 + (position_it[1] - x[1][1])**2))
                        #print(position_it, closest_last_position)
                        Rus[closest_last_position[0], param_idx] = position_it[0]
                        Ros[closest_last_position[0], param_idx] = position_it[1]
                        prop_Ro[closest_last_position[0], param_idx] = position_it[1]/np.sum(position_it) 
                        sum_Rs[closest_last_position[0], param_idx] = np.sum(position_it)
                        stab, eig = stabFuncs.jacobian_eignvalues((position_it[0], position_it[1]), para)
                        stabs[closest_last_position[0], param_idx] = stab #Stability function that can handle nans
                        if position_it[0] > 0 and stab:
                            stable_nontriv[closest_last_position[0], param_idx] = 1
                        if position_it[0] == 0 and stab:
                            stable_bloat[closest_last_position[0], param_idx] = 1
                        output = baseFuncs.behavioral_work_eqns((position_it[0], position_it[1]), para, True, True)
                        prop_w_A[closest_last_position[0], param_idx] = output[0]/para['w_max']
                        prop_w_A_star[closest_last_position[0], param_idx] = output[7]/para['w_max']
                        ps[closest_last_position[0], param_idx] = output[6]
                        utilities[closest_last_position[0], param_idx] = para['A']*((output[1])**(para['alpha']))*((position_it[0])**(para['beta']))

        # Stability classifications
        non_zero_non_nan = np.count_nonzero((stabs[:, param_idx])) - np.sum(np.isnan(stabs[:, param_idx]))
        stab_cat.append((non_zero_non_nan, len(stabs[:, param_idx])-np.count_nonzero(stabs[:, param_idx])))
        
    output = {"param_vary" : param_vary, "param_label" : param_label, "param_positions" : param_positions, "stab_cat_Rs" : stab_cat_Rs, 
              "sum_Rs" : sum_Rs, "Rus" : Rus, "Ros" : Ros, "branch_num" : branch_num, "stabs" : stabs, "utilities" : utilities,
              "stab_cat" : stab_cat, "eigs_list" : eigs_list, "prop_w_A" : prop_w_A, "prop_Ro" : prop_Ro, "stable_nontriv" : stable_nontriv,
              "ps" : ps, "prop_w_A_star": prop_w_A_star, "para": para, "stable_bloat" : stable_bloat}

    if stab_slice:
        return stab_cat, stab_cat_Rs, eigs_list
    else:
        return output

In [5]:
para = baseFuncs.paraReset()
param_label = "dtilde"
lower = 0.01
upper = 1
data1 = paramSlice(param_label, lower, upper, para, param_mesh = 51, init_mesh = 15, xtol = 1e-6, stab_slice = False)

### Slice along two parameters

In [3]:
def evaluatePlane(para, param_x_label, param_y_label, bnds, param_x_mesh = 50, param_y_mesh = 50, init_mesh = 15):
    """
    Function to evaluate the stability qualities for points over the 2D space of (param_x, param_y)
    bnds is a tuple of the form ((param_x_lower, param_x_upper), (param_y_lower, param_y_upper))
    """
    
    # Axes and Grid
    param_xs = np.linspace(bnds[0][0], bnds[0][1], param_x_mesh)
    param_ys = np.linspace(bnds[1][0], bnds[1][1], param_y_mesh)
    param_x_grid, param_y_grid = np.meshgrid(param_xs, param_ys)
    stab_grid = np.ones([param_y_mesh, param_x_mesh])
    stab_grid[:] = 0

    # Stability slice for each x value
    for i, param_x_val in enumerate(param_xs):
        para[param_x_label] = param_x_val
        output_dict = paramSlice(param_y_label, bnds[1][0], bnds[1][1], para, param_mesh = param_y_mesh, stab_slice = False, init_mesh = init_mesh)
        stab_nontriv = output_dict['stable_nontriv']
        stab_bloat = output_dict['stable_bloat']
        
        # Classify each point along the slice
        for k, param_y_val in enumerate(param_ys):
            para[param_y_label] = param_y_val
            stab_grid[k,i] += (1*any(stab_nontriv[:,k]) + 2*any(stab_bloat[:, k]))
    
    output = {"param_x_label" : param_x_label, "param_y_label" : param_y_label, "param_xs" : param_xs, "param_ys" : param_ys, 
             "param_x_grid" : param_x_grid, "param_y_grid" : param_y_grid, "stab_grid" : stab_grid}
    
    return output

In [29]:
def evaluatePerformancePlane(para, param_x_label, param_y_label, bnds, param_x_mesh = 50, param_y_mesh = 50, init_mesh = 15):
    """
    Function to evaluate the stability qualities for points over the 2D space of (param_x, param_y)
    bnds is a tuple of the form ((param_x_lower, param_x_upper), (param_y_lower, param_y_upper))
    """
    
    # Axes and Grid
    param_xs = np.linspace(bnds[0][0], bnds[0][1], param_x_mesh)
    param_ys = np.linspace(bnds[1][0], bnds[1][1], param_y_mesh)
    param_x_grid, param_y_grid = np.meshgrid(param_xs, param_ys)
    Ru_grid = np.ones([param_y_mesh, param_x_mesh])
    Ru_grid[:] = 0
    Ro_grid = np.ones([param_y_mesh, param_x_mesh])
    Ro_grid[:] = 0
    prop_wA_grid = np.ones([param_y_mesh, param_x_mesh])
    prop_wA_grid[:] = 0
    prop_Ro_grid = np.ones([param_y_mesh, param_x_mesh])
    prop_Ro_grid[:] = 0
    utility_grid = np.ones([param_y_mesh, param_x_mesh])
    utility_grid[:] = 0

    # Stability slice for each x value
    for i, param_x_val in enumerate(param_xs):
        if param_x_label == "employees":
            para = baseFuncs.paraUpdateEmployees(para, employees = param_x_val)
        else:
            para[param_x_label] = param_x_val
        output_dict = paramSlice(param_y_label, bnds[1][0], bnds[1][1], para, param_mesh = param_y_mesh, stab_slice = False, init_mesh = init_mesh)
        stab_nontriv = output_dict['stable_nontriv']
        stab_bloat = output_dict['stable_bloat']
        
        # Classify each point along the slice
        Ru_grid[:,i] = np.select(output_dict['stable_nontriv'].astype(bool), output_dict['Rus'], 0)
        Ro_grid[:,i] = np.select(output_dict['stable_nontriv'].astype(bool), output_dict['Ros'], output_dict['Ros'][0])
        prop_wA_grid[:,i] = np.select(output_dict['stable_nontriv'].astype(bool), output_dict['prop_w_A'], 1)
        prop_Ro_grid[:,i] = np.select(output_dict['stable_nontriv'].astype(bool), output_dict['prop_Ro'], 1)
        utility_grid[:,i] = np.select(output_dict['stable_nontriv'].astype(bool), output_dict['utilities'], 0)
    
    output = {"param_x_label" : param_x_label, "param_y_label" : param_y_label, "param_xs" : param_xs, "param_ys" : param_ys, 
             "param_x_grid" : param_x_grid, "param_y_grid" : param_y_grid, "Ru_grid" : Ru_grid, "Ro_grid" : Ro_grid, 
             "prop_wA_grid" : prop_wA_grid, "prop_Ro_grid" : prop_Ro_grid, "utility_grid" : utility_grid}
    
    return output

### Plotting the Slice

In [4]:
def plotParamSlice(data, plot_type = 1, show_rules = True, show_sum = False, saveFig = False, saveName = None, log_y = False):
    # Load data
    branch_num = data['branch_num']
    stabs = data['stabs']
    param_vary = data['param_vary']
    Rus = data['Rus']
    Ros = data['Ros']
    sum_Rs = data['sum_Rs']
    param_label = data['param_label']
    prop_w_A = data['prop_w_A']
    prop_Ro = data['prop_Ro']
    ps = data['ps']
    prop_w_A_star = data['prop_w_A_star']
    
    # Plotting the whole thing
    if plot_type == 1:
        fig, ax1 = plt.subplots(1,1)
    elif plot_type == 2:
        fig, (ax1, ax2) = plt.subplots(1,2, figsize = (9,4))
    elif plot_type == 3:
        fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (13,4))
    else:
        raise ValueError("Please ensure that plot type is either 1 or 2")
        # I considered puting the tildes in here, but decided to not do that yet because it might be too much with the multiple branches.
    if log_y:
        ax1.set_yscale('log')
        Rus = Rus + 1
        Ros = Ros + 1
        
    for i in range(branch_num):
        stabIdx = (stabs[i,:]==1)
        unstabIdx = (stabs[i,:]==0)
        if show_rules:
            ax1.plot(param_vary[stabIdx], Rus[i, stabIdx], linestyle = "-", color = "blue")
            ax1.plot(param_vary[stabIdx], Ros[i, stabIdx], linestyle = "-", color = "orange")
            ax1.plot(param_vary[unstabIdx], Rus[i, unstabIdx], linestyle = "--", color = "blue", linewidth = 2.5)
            ax1.plot(param_vary[unstabIdx], Ros[i, unstabIdx], linestyle = "--", color = "orange", linewidth = 2.5)
            
            if show_sum:
                ax1.plot(param_vary[stabIdx], sum_Rs[i, stabIdx], "k-")
                ax1.plot(param_vary[unstabIdx], sum_Rs[i, unstabIdx], "k--")
            ax1.set_ylabel("Rules", fontsize = 14)
            ax1.set_xlim([param_vary[0], param_vary[-1]])
            ax1.legend([r"$R_u$", r"$R_o$"])
            
        if plot_type == 2 or plot_type == 3:      
            ax2.set_ylabel(r"Admin Metrics", fontsize = 14)
            if i == 0:
                ax2.plot(param_vary[stabIdx], prop_Ro[i, stabIdx], "k-", label = r"$R_o/R_u+R_o$")    
            ax2.plot(param_vary[stabIdx], prop_Ro[i, stabIdx], "k-")
            ax2.plot(param_vary[unstabIdx], prop_Ro[i, unstabIdx], "k--")
            ax2.set_ylim([-0.05,1.05])
            ax2.set_xlim([param_vary[0], param_vary[-1]])
            
            if i == 0:
                ax2.plot(param_vary[stabIdx], prop_w_A[i, stabIdx], "-", color = "blue", label = r"$w_A/w$")
            ax2.plot(param_vary[stabIdx], prop_w_A[i, stabIdx], "-", color = "blue")
            ax2.plot(param_vary[unstabIdx], prop_w_A[i, unstabIdx], "--", color = "blue")
            ax2.legend([r"$R_o/(R_o+R_u)$", r"$w_A/w$"], loc = "best")
            leg = ax2.get_legend()
            leg.legend_handles[0].set_color('black')
            leg.legend_handles[1].set_color('blue')
        if plot_type == 3:
            ax3.plot(param_vary[stabIdx], ps[i, stabIdx], linestyle = "-", color = "black")
            ax3.plot(param_vary[unstabIdx], ps[i, unstabIdx], linestyle = "--", color = "black")
            ax3.set_ylabel("ptilde", fontsize = 14)
            ax3.set_ylim([-0.05, 1.05])
            ax3.set_xlim([param_vary[0], param_vary[-1]])
    
    if param_label == 'gamma_c' or param_label == 'gamma_p':
        fig.suptitle("Bifurcation Plot of Equil. Rules vs "+rf"$\{param_label}$", fontsize = 14) 
        ax1.set_xlabel(fr"$\{param_label}$", fontsize = 14)
        if plot_type >= 2:
            ax2.set_xlabel(fr"$\{param_label}$", fontsize = 14)
        if plot_type >= 3:
            ax3.set_xlabel(fr"$\{param_label}$", fontsize = 14)
    else:
        fig.suptitle("Bifurcation Plot of Equil. Rules vs "+rf"${param_label}$", fontsize = 14) 
        ax1.set_xlabel(fr"${param_label}$", fontsize = 14)
        if plot_type >= 2:
            ax2.set_xlabel(fr"${param_label}$", fontsize = 14)
        if plot_type >= 3:
            ax3.set_xlabel(fr"${param_label}$", fontsize = 14)
    
    plt.subplots_adjust(wspace=0.25)
    if saveFig:
        plt.savefig(saveName, dpi=500, bbox_inches = "tight")
    else:
        plt.show()

## Plotting the Plane

In [5]:
def plotPlanePanel(ax, data, plot_legend = False, plot_labels = [(True, "xlabel"), (True, "ylabel")], fontsize = 12, title = None):
    # Load data
    param_x_grid = data['param_x_grid']
    param_y_grid = data['param_y_grid']
    stab_grid = data['stab_grid']
    
    # Colormap
    clist = []
    handles = []
    hatches = []
    patch1 = mpatches.Patch(color='#C79FEF', label='Unbounded Bloat')
    patch2 = mpatches.Patch(color='#EAB676', label='Sustainable Equilibrium\n and Bloat')
    patch3 = mpatches.Patch(color='#C79FEF', label='Bounded Bloat')
    patch4 = mpatches.Patch(color='#EAB676', label='Sustainable Equilibrium\n and Bloat')

    cmap_prelist = [['#C79FEF', patch1, None], ['#EAB676', patch2, None], ['#C79FEF', patch3, None], ['#EAB676', patch4, None]]
    uniq_vals = np.unique(stab_grid)
    for val_it in uniq_vals:
        clist.append(cmap_prelist[int(val_it)][0])
        handles.append(cmap_prelist[int(val_it)][1])
        hatches.append(cmap_prelist[int(val_it)][2])
    num_levels = len(uniq_vals) + 1
    
    cmap = mpl.colors.ListedColormap(clist)
    bounds = list(np.linspace(0, 1, num_levels))
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
        
    cp = ax.contourf(param_x_grid, param_y_grid, stab_grid, cmap = cmap, levels = num_levels, hatches=hatches)
    if plot_legend:
        ax.legend(handles=handles, framealpha = 1, loc = 'upper left', bbox_to_anchor=(1, 1))

    ax.set_xlim(min(data['param_xs']), max(data['param_xs']))
    ax.set_ylim(min(data['param_ys']), max(data['param_ys']))

    if title is not None:
        ax.set_title(title, fontsize = fontsize, weight = "bold")
    
    if plot_labels[0][0]:
        ax.set_xlabel(plot_labels[0][1], fontsize = fontsize)
    if plot_labels[1][0]:
        ax.set_ylabel(plot_labels[1][1], fontsize = fontsize)